In [ ]:
import os
import pickle
import json
import random
from tqdm import tqdm

from openai import OpenAI
import time
client = OpenAI(api_key="xxx")

In [ ]:
#domain="Materials_Science" # Medicine, Chemistry, Biology, Physics, Materials_Science
#data_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/processed_data/maple/{domain}"
#downstream_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/raw_data/maple"

#data_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/processed_data/biomedical"
#downstream_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/raw_data/biomedical"

data_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/processed_data/dblp"
downstream_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/raw_data/dblp"

#data_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/processed_data/amazon"
#downstream_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/raw_data/amazon"

#data_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/processed_data/goodreads"
#downstream_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/raw_data/goodreads"

#data_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/processed_data/legal"
#downstream_dir=f"/shared/data3/bowenj4/llm-graph-plugin/data/raw_data/legal"

## Obtain question-answer samples

In [ ]:
processed_data = pickle.load(open(os.path.join(data_dir, 'preprocess_samples.pkl'), 'rb'))

### We prompt gpt4 to get more diverse templates for each question type

In [ ]:
paraphrase_prompt = "Paraphrase the given template in four different ways. Keep the name in '{}' unchanged, don't use ' in question and use the same format ('question_string', 'answer_string'): "

def paraphrase_gpt4(qa_template, k=5):
    x = paraphrase_prompt + '\n' + qa_template
    response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": x},
            ]
        )
    para_list = response.choices[0].message.content.split('\n')
    para_list.append(qa_template)
    filter_para_list = []
    for p in para_list:
        try:
            eval(p)
            if len(p) != 0:
                if p[-1] == ',':
                    p = p[:-1]
                filter_para_list.append(p)
        except:
            pass
    
    return filter_para_list

res = paraphrase_gpt4("('How many diseases have the same symptom with {disease_name}?', '{num}')")
print(res)

In [ ]:
random.seed(2023)

qid = 0

with open(os.path.join(data_dir, 'data.json'), 'w') as fout:
    for qa_template in tqdm(processed_data):
        # q_template, a_template = qa_template
        paraphrase_qa_templates = paraphrase_gpt4(str(qa_template))
        for sample in processed_data[qa_template]:
            try:
                randc = random.choice(paraphrase_qa_templates)
                q_template, a_template = eval(randc)
            except:
                try:
                    randc = random.choice(paraphrase_qa_templates)
                    q_template, a_template = eval(randc)
                except:
                    randc = random.choice(paraphrase_qa_templates)
                    q_template, a_template = eval(randc)
            tmp_question = q_template.format(**sample)
            tmp_answer = a_template.format(**sample)
            fout.write(json.dumps({
                'qid': str(qid),
                'question': tmp_question,
                'answer': tmp_answer
            }) + '\n')
            qid += 1

In [ ]:
paraphrase_qa_templates

In [ ]:
sample